

### Business scenario

You work for a training organization that recently developed an introductory course about machine learning (ML). The course includes more than 40 videos that cover a broad range of ML topics. You have been asked to create an application that will students can use to quickly locate and view video content by searching for topics and key phrases.

You have downloaded all of the videos to an Amazon Simple Storage Service (Amazon S3) bucket. Your assignment is to produce a dashboard that meets your supervisor’s requirements.

## Project steps

To complete this project the required steps are:

1. [Viewing the video files](#1.-Viewing-the-video-files)
2. [Transcribing the videos](#2.-Transcribing-the-videos)
3. [Normalizing the text](#3.-Normalizing-the-text)
4. [Extracting key phrases and topics](#4.-Extracting-key-phrases-and-topics)
5. [Creating the dashboard](#5.-Creating-the-dashboard)

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
bucket = os.getenv("BUCKET")
job_data_access_role = os.getenv("JOB_DATA_ACCESS_ROLE")
source_bucket = os.getenv("SOURCE_BUCKET")
source_prefix = os.getenv("SOURCE_PREFIX")

## 1. Viewing the video files
([Go to top](#Capstone-8:-Bringing-It-All-Together))


The source video files are located in the following shared Amazon Simple Storage Service (Amazon S3) bucket.

In [2]:
!aws s3 ls s3://{source_bucket}/{source_prefix}

2021-04-26 20:17:33  410925369 Mod01_Course Overview.mp4
2021-04-26 20:10:02   39576695 Mod02_Intro.mp4
2021-04-26 20:31:23  302994828 Mod02_Sect01.mp4
2021-04-26 20:17:33  416563881 Mod02_Sect02.mp4
2021-04-26 20:17:33  318685583 Mod02_Sect03.mp4
2021-04-26 20:17:33  255877251 Mod02_Sect04.mp4
2021-04-26 20:23:51   99988046 Mod02_Sect05.mp4
2021-04-26 20:24:54   50700224 Mod02_WrapUp.mp4
2021-04-26 20:26:27   60627667 Mod03_Intro.mp4
2021-04-26 20:26:28  272229844 Mod03_Sect01.mp4
2021-04-26 20:27:06  309127124 Mod03_Sect02_part1.mp4
2021-04-26 20:27:06  195635527 Mod03_Sect02_part2.mp4
2021-04-26 20:28:03  123924818 Mod03_Sect02_part3.mp4
2021-04-26 20:31:28  171681915 Mod03_Sect03_part1.mp4
2021-04-26 20:32:07  285200083 Mod03_Sect03_part2.mp4
2021-04-26 20:33:17  105470345 Mod03_Sect03_part3.mp4
2021-04-26 20:35:10  157185651 Mod03_Sect04_part1.mp4
2021-04-26 20:36:27  187435635 Mod03_Sect04_part2.mp4
2021-04-26 20:36:40  280720369 Mod03_Sect04_part3.mp4
2021-04-26 20:40:01  443479

## 2. Transcribing the videos
 ([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to implement your solution to transcribe the videos. 

In [3]:
!pip install --upgrade pip
!pip install moviepy torch transformers
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate
!pip install nltk
!pip install spacy
!pip install python-dotenv
!python -m spacy download en_core_web_sm

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-cu667638
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-cu667638
  Resolved https://github.com/huggingface/transformers.git to commit b109257f4fb8b1166e7c53cc5418632014ed53a5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import os
import pandas as pd
import boto3
from moviepy.editor import VideoFileClip
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# Directory to save audios
directory = "audios"

# Initialize the S3 client
s3 = boto3.client("s3")


# Convert videos to wav audio files
def process_video(video_key):
    try:
        # Check if the directory exists
        if not os.path.exists(directory):
            # Create directory if it doesn"t exist
            os.makedirs(directory)

        # Extract the video ID
        video_id = video_key.split("/")[-1].replace(".mp4", "")

        # Output audio
        audio_file = f"{directory}/{video_id}.wav"

        # Download the video to a temporary file
        tmp_video_file = "/tmp/video.mp4"
        s3.download_file(source_bucket, video_key, tmp_video_file)

        # Load the video clip
        video_clip = VideoFileClip(tmp_video_file)

        # Extract audio from the video clip
        audio_clip = video_clip.audio

        # Write the audio to a 16 bit wav file
        audio_clip.write_audiofile(audio_file, codec="pcm_s16le")

        # Close the video clip
        video_clip.close()
        print(f"Successfully processed {video_key}")


    except Exception as e:
        print(f"Error processing {video_key}: {e}")
        return None

# Convert audio to text
def transcribe_audio(audio_path):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    model_id = "openai/whisper-large-v3"
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)

    processor = AutoProcessor.from_pretrained(model_id)

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        max_new_tokens=128,
        chunk_length_s=30,
        batch_size=16,
        return_timestamps=True,
        torch_dtype=torch_dtype,
        device=device,
    )

    result = pipe(audio_path)
    print(result["text"])
    return result["text"]


# List objects in the source bucket
response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_prefix)

# Create
df = pd.DataFrame(columns=["video", "audio", "text"])

# Process each video
if "Contents" in response:
    for obj in response["Contents"]:
        video_key = obj["Key"]
        if video_key.endswith(".mp4"):
            process_video(video_key)
            video_name = video_key.split("/")[-1]
            audio_name = video_name.replace(".mp4", ".wav")
            audio_text = transcribe_audio(f"{directory}/{audio_name}")
            new_df = pd.DataFrame([{"video": video_name, "audio": audio_name, "text": audio_text}])
            df = pd.concat([df, new_df], ignore_index=True)

df

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5033:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2501:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4554:(_snd_config_evaluate) function snd_func_concat returned error: N

MoviePy - Writing audio in audios/Mod01_Course Overview.wav


MoviePy - Done.
Successfully processed CUR-TF-200-ACMNLP-1/video/Mod01_Course Overview.mp4


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Hi, and welcome to Amazon Academy Machine Learning Foundations. In this module, you'll learn about the course objectives, various job roles in the machine learning domain, and where you can go to learn more about machine learning. After completing this module, you should be able to identify course prerequisites and objectives, indicate the role of the data scientist in business, and identify resources for further learning. We're now going to look at the prerequisites for taking this course. Before you take this course, we recommend that you first complete AWS Academy Cloud Foundations. You should also have some general technical knowledge of IT, including foundational computer literacy skills like basic computer concepts, email, file management, and a good understanding of the Internet. We also recommend that you have intermediate skills with Python programming and a general knowledge of applied statistics. Finally, general business knowledge is important for this course. This include

...

 Welcome back. It's now time to review the module and wrap it up. In summary, in this module, you learned how to describe the NLP use cases that are solved by using managed Amazon ML services and describe the managed ML services available for NLP. Good job. Thanks for watching. We'll see you in the next module. and describe the managed ML services available for NLP. Good job. Thanks for watching. We'll see you in the next module.
MoviePy - Writing audio in audios/Mod07_Sect01.wav


MoviePy - Done.
Successfully processed CUR-TF-200-ACMNLP-1/video/Mod07_Sect01.mp4


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Welcome to Module 7, Course Wrap-Up. Congratulations on completing the AWS Academy Machine Learning course. We'll take a few minutes to review what you've learned and where you can go from here. We're going to start with a review of what you've learned in this course. You learned how to describe machine learning, implement a machine learning pipeline, and use Amazon machine learning services for forecasting, computer vision, and natural language processing. Well done. Although this course isn't designed to prepare you to become certified for the AWS Certified Machine Learning specialty, we'll review how you can continue to work towards that certification. AWS Certification helps you build credibility and confidence by validating your cloud expertise with an industry-recognized credential. It also helps organizations identify skilled professionals who can lead cloud initiatives by using AWS. You must earn a passing score by taking a proctored exam to earn an AWS certification. After re

,video,audio,text
0,Mod01_Course Overview.mp4,Mod01_Course Overview.wav,"Hi, and welcome to Amazon Academy Machine Lea..."
1,Mod02_Intro.mp4,Mod02_Intro.wav,"Hi, and welcome to Module 2 of AWS Academy Ma..."
2,Mod02_Sect01.mp4,Mod02_Sect01.wav,"Hi, and welcome to Section 1. In this section..."
3,Mod02_Sect02.mp4,Mod02_Sect02.wav,"Hi and welcome back. In this section, we're g..."
4,Mod02_Sect03.mp4,Mod02_Sect03.wav,Hi and welcome back. This is section 3 and we...
5,Mod02_Sect04.mp4,Mod02_Sect04.wav,"Welcome back. In this section, we'll look at ..."
6,Mod02_Sect05.mp4,Mod02_Sect05.wav,"Hi, welcome back. This is section 5 and we're..."
7,Mod02_WrapUp.mp4,Mod02_WrapUp.wav,It's now time to review the module. Here are ...
8,Mod03_Intro.mp4,Mod03_Intro.wav,Welcome back to AWS Academy Machine Learning....
9,Mod03_Sect01.mp4,Mod03_Sect01.wav,"Hi, and welcome back to module 3. This is sec..."


## 3. Normalizing the text
([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to perform any text normalization steps that are necessary for your solution.

In [5]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re


# Text normalization and lemmatization
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub("[^A-Za-z0-9]+", " ", text)

    # Replace all newline characters (\n) with spaces
    text = re.sub(r"\n", " ", text)

    # Remove leading or trailing whitespaces
    text = text.strip()

    # Remove stopwords
    stop_words = set(stopwords.words("english"))

    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = " ".join(tokens)
    return tokens

df["normalized_text"] = df["text"].apply(preprocess_text)
df

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,video,audio,text,normalized_text
0,Mod01_Course Overview.mp4,Mod01_Course Overview.wav,"Hi, and welcome to Amazon Academy Machine Lea...",hi welcome amazon academy machine learning fou...
1,Mod02_Intro.mp4,Mod02_Intro.wav,"Hi, and welcome to Module 2 of AWS Academy Ma...",hi welcome module aws academy machine learning...
2,Mod02_Sect01.mp4,Mod02_Sect01.wav,"Hi, and welcome to Section 1. In this section...",hi welcome section section going talk machine ...
3,Mod02_Sect02.mp4,Mod02_Sect02.wav,"Hi and welcome back. In this section, we're g...",hi welcome back section going look type busine...
4,Mod02_Sect03.mp4,Mod02_Sect03.wav,Hi and welcome back. This is section 3 and we...,hi welcome back section going give quick high ...
5,Mod02_Sect04.mp4,Mod02_Sect04.wav,"Welcome back. In this section, we'll look at ...",welcome back section look tool using throughou...
6,Mod02_Sect05.mp4,Mod02_Sect05.wav,"Hi, welcome back. This is section 5 and we're...",hi welcome back section going discus challenge...
7,Mod02_WrapUp.mp4,Mod02_WrapUp.wav,It's now time to review the module. Here are ...,time review module main takeaway module first ...
8,Mod03_Intro.mp4,Mod03_Intro.wav,Welcome back to AWS Academy Machine Learning....,welcome back aws academy machine learning modu...
9,Mod03_Sect01.mp4,Mod03_Sect01.wav,"Hi, and welcome back to module 3. This is sec...",hi welcome back module section going take look...


## 4. Extracting key phrases and topics
([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to extract the key phrases and topics from the videos.

In [6]:
import spacy

# Load spacy's English NLP model
nlp = spacy.load("en_core_web_sm")

# Extract key phrases
def extract_keyphrases(text):
    doc = nlp(text)
    return [chunk.text for chunk in doc.noun_chunks]

df["key_phrases"] = df["normalized_text"].apply(extract_keyphrases)

# Extract topics
labels = {"LANGUAGE", "ORDINAL", "EVENT", "PRODUCT", "DATE", "WORK_OF_ART", "FAC",  "ORG", "PERSON",  "GPE"}

def extract_topics(text):
    doc = nlp(text)

    # Extract entities
    topics = [entity.text for entity in doc.ents if entity.label_ in labels]

    # Extract all nouns
    topics.extend([token.text for token in doc if token.pos_ == "NOUN"])
    return topics

df["topics"] = df["normalized_text"].apply(extract_topics)
df


,video,audio,text,normalized_text,key_phrases,topics
0,Mod01_Course Overview.mp4,Mod01_Course Overview.wav,"Hi, and welcome to Amazon Academy Machine Lea...",hi welcome amazon academy machine learning fou...,"[amazon academy machine, foundation module, co...","[machine learning, first, amazon, lex amazon, ..."
1,Mod02_Intro.mp4,Mod02_Intro.wav,"Hi, and welcome to Module 2 of AWS Academy Ma...",hi welcome module aws academy machine learning...,"[introduce machine, first look business proble...","[first, welcome, module, machine, module, intr..."
2,Mod02_Sect01.mp4,Mod02_Sect01.wav,"Hi, and welcome to Section 1. In this section...",hi welcome section section going talk machine ...,"[welcome section section, talk machine learnin...","[first, every year, first, welcome, section, s..."
3,Mod02_Sect02.mp4,Mod02_Sect02.wav,"Hi and welcome back. In this section, we're g...",hi welcome back section going look type busine...,"[section, machine learning, digital life email...","[anomaly x ray, first, boundary cross, first, ..."
4,Mod02_Sect03.mp4,Mod02_Sect03.wav,Hi and welcome back. This is section 3 and we...,hi welcome back section going give quick high ...,"[section, quick high level overview machine, t...","[november, february year, mali city, age birth..."
5,Mod02_Sect04.mp4,Mod02_Sect04.wav,"Welcome back. In this section, we'll look at ...",welcome back section look tool using throughou...,"[section look tool, rest course, exhaustive li...","[today, first, linear, tensorflow kera, first,..."
6,Mod02_Sect05.mp4,Mod02_Sect05.wav,"Hi, welcome back. This is section 5 and we're...",hi welcome back section going discus challenge...,"[section, discus challenge machine learning, m...","[today, first, recognition, section, discus, c..."
7,Mod02_WrapUp.mp4,Mod02_WrapUp.wav,It's now time to review the module. Here are ...,time review module main takeaway module first ...,"[time review module main takeaway module, land...","[time, review, module, module, machine, fit, l..."
8,Mod03_Intro.mp4,Mod03_Intro.wav,Welcome back to AWS Academy Machine Learning....,welcome back aws academy machine learning modu...,"[back aws academy machine learning module, ama...","[machine, module, work, machine, pipeline, mod..."
9,Mod03_Sect01.mp4,Mod03_Sect01.wav,"Hi, and welcome back to module 3. This is sec...",hi welcome back module section going take look...,"[module section, data set use module, guidance...","[first, first, first, six month, first, second..."


## 5. Creating the dashboard
([Go to top](#Capstone-8:-Bringing-It-All-Together))

Use this section to create the dashboard for your solution.

In [7]:
from ipywidgets import Label, widgets
from IPython.display import display, HTML

# Create a textbox for the key phrases and topics and label for it
textbox = widgets.Text(description='Search:')
textbox_label = Label(value='Enter a keyword or topic to search videos:')

display(textbox_label, textbox)

# Search videos based on keywords and topics
def search_video(sender):
    # Clear the earlier output
    output.clear_output()
    term = sender.value.strip()

    query = term.lower()
    search_result = 0

    if query != '':
        with output:
            for index, row in df.iterrows():
                if any(query in phrase for phrase in row['key_phrases']) or any(query in topic for topic in row['topics']):
                    search_result += 1
                    video_name = row['video']
                    video_url = f"https://{source_bucket}.s3.amazonaws.com/{source_prefix}{video_name}"
                    display(HTML(f"</br><p><b>{video_name.replace('.mp4','')}</b></p><video width='400' controls><source src='{video_url}' type='video/mp4'></video>"))
            if search_result >0:
                print(f"Found {search_result} video(s) related to '{term}'.")

        if search_result == 0:
            with output:
                print(f"No video found related to '{term}'.")


textbox.on_submit(search_video)

# Create an output widget to display the search results
output = widgets.Output()
display(output)


Label(value='Enter a keyword or topic to search videos:')

Text(value='', description='Search:')

Output()